In [1]:
import sidechainnet as scn
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from sidechainnet.dataloaders.SCNProtein import atom_name_pprint
from sidechainnet.tests.test_hydrogens import load_p, get_alias
from sidechainnet.structure import fastbuild
from sidechainnet.structure import StructureBuilder
from torch.autograd import gradcheck
from torch.autograd.functional import jacobian as get_jacobian
torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(sci_mode=False, precision=3)
np.set_printoptions(precision=3)
torch.cuda.current_device(), torch.cuda.is_available()

(0, True)

In [2]:
import importlib
importlib.reload(fastbuild)

<module 'sidechainnet.structure.fastbuild' from '/home/jok120/sidechainnet/sidechainnet/structure/fastbuild.py'>

In [7]:
d = scn.load(12, 100, local_scn_path='/home/jok120/sidechainnet_data/sidechainnet_debug.pkl',
scn_dir='/home/jok120/sidechainnet_data',
  scn_dataset=True,
  complete_structures_only=True)
# p = d['70#4MMO_1_A']
p = d['1A38_2_P']
# p = d[1]
p

SidechainNet was loaded from /home/jok120/sidechainnet_data/sidechainnet_debug.pkl.
27 (5.5%) data set entries were excluded due to missing residues.


SCNProtein(1A38_2_P, len=5, missing=0, split='train')

In [8]:
p.sb = None
p.to_3Dmol()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
# p.angles[0, 6] = - 3/2 * np.pi

p.sb = None
p.to_3Dmol(from_angles=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
%%time
p.sb = None
p.build_coords_from_angles(p.angles)

CPU times: user 194 ms, sys: 8.09 ms, total: 202 ms
Wall time: 201 ms


array([[ 0.000e+00,  0.000e+00,  1.000e-03],
       [ 1.442e+00,  0.000e+00,  1.000e-03],
       [ 1.923e+00,  1.419e+00,  1.000e-03],
       [ 1.125e+00,  2.338e+00, -1.635e-01],
       [ 1.956e+00,  0.000e+00,  1.000e-03],
       [ 2.602e+00,  0.000e+00,  1.000e-03],
       [ 3.378e+00,  0.000e+00,  1.000e-03],
       [ 3.821e+00,  0.000e+00,  1.000e-03],
       [ 4.329e+00,  0.000e+00,  1.000e-03],
       [ 5.280e+00,  0.000e+00,  1.000e-03],
       [ 5.980e+00,  0.000e+00,  1.000e-03],
       [ 6.680e+00,  0.000e+00,  1.000e-03],
       [ 7.380e+00,  0.000e+00,  1.000e-03],
       [ 8.082e+00,  0.000e+00,  1.000e-03],
       [ 3.271e+00,  1.640e+00,  1.896e-01],
       [ 3.840e+00,  2.965e+00,  2.066e-01],
       [ 4.991e+00,  3.149e+00, -7.343e-01],
       [ 5.132e+00,  2.388e+00, -1.689e+00],
       [ 4.306e+00,  3.307e+00,  1.619e+00],
       [ 5.644e+00,  2.627e+00,  1.892e+00],
       [ 6.327e+00,  3.304e+00,  3.077e+00],
       [ 5.421e+00,  1.133e+00,  2.111e+00],
       [  

In [ ]:
%%time
p.sb = None
p.build_coords_from_angles(p.angles,add_hydrogens=True)

In [ ]:
%%time
ang2 = torch.Tensor(p.angles)
ang2[0, 6] = 0  # Make arbitrary CB definition
sb = StructureBuilder(p.seq, ang=ang2,device=p.device)
goodcoords = sb.build()

In [ ]:
p.seq

In [ ]:
from sidechainnet.structure.build_info import SC_BUILD_INFO, BB_BUILD_INFO, NUM_COORDS_PER_RES, SC_ANGLES_START_POS, NUM_ANGLES

def make_ncac(p,device='cpu'):
    '''For testing make_backbone'''
    L = len(p.seq)
    angles = p.angles
    ang = torch.zeros((L+1,SC_ANGLES_START_POS),device=device)
    ang[1:] = angles.to(device)[:L,:SC_ANGLES_START_POS] 
    ang[:,3:] = np.pi-ang[:,3:] #theta
    ang[:,:3] = 2*np.pi-ang[:,:3] #chi

    sins = torch.sin(ang)
    coss = torch.cos(ang)
    #O needs one additional angle
    schi = sins[:,:3].flatten()[1:-1]
    cchi = coss[:,:3].flatten()[1:-1]

    oschi = schi[3::3]
    occhi = cchi[3::3]
    schi = schi[:-1]
    cchi = cchi[:-1]

    stheta = sins[:,3:].flatten()[1:-2]
    ctheta = coss[:,3:].flatten()[1:-2]

    lens = torch.tensor([BB_BUILD_INFO['BONDLENS']['c-n'],BB_BUILD_INFO['BONDLENS']['n-ca'],BB_BUILD_INFO['BONDLENS']['ca-c']],device=device).repeat(L)
    lens[0] = 0 # first atom starts at origin
    ncacM = fastbuild.makeTmats(ctheta, stheta, cchi, schi, lens)

    return ncacM

In [ ]:
p.torch()

In [ ]:
fastbuild.sc_heavy_atom_build_info['CA']['sources'].dtype

In [ ]:
importlib.reload(fastbuild)

angles = p.angles.detach().clone()
print(angles)
angles[0, 0] = 0
angles[0, 6] = -2/3 * np.pi
genc = fastbuild.make_coords(p.seq,angles)

In [ ]:
diff = (genc.cpu()-goodcoords.reshape(genc.shape)).reshape((len(p.seq),-1))
torch.nansum(diff*diff)

In [ ]:
plt.figure(figsize=(6,12))
sns.heatmap(diff.detach().numpy()[:50],yticklabels=p.seq[:50],cmap='seismic',vmin=-2,vmax=2)

In [ ]:
14*3

In [ ]:
p.to_3Dmol()

In [ ]:
%%timeit
angles.requires_grad = False
genc = fastbuild.make_coords(p.seq,angles)

In [ ]:
%%timeit
angles.requires_grad = True
genc = fastbuild.make_coords(p.seq,angles)

In [ ]:
angles.requires_grad = False
angles = angles.to('cuda')

In [ ]:
%%timeit
genc = fastbuild.make_coords(p.seq,angles)

In [ ]:
angles.requires_grad = True
angles = angles.to('cuda')

In [ ]:
%%timeit
genc = fastbuild.make_coords(p.seq,angles)

In [ ]:
p.torch()

In [ ]:
p.angles.requires_grad = True

In [ ]:
sb = StructureBuilder(p.seq, ang=p.angles,device=p.device,nerf_method='standard')

In [ ]:
%%timeit
goodcoords = sb.build()

In [ ]:
%%time
sb.add_hydrogens()

In [ ]:
ncac = make_ncac(p)

In [ ]:
smn = ncac.clone().detach()
smn.requires_grad = True
%time backbone = fastbuild.make_backbone_(smn.clone())
s = backbone.sum()
%time s.backward()

In [ ]:
goodgrad = smn.grad

In [ ]:
smn = ncac.clone().detach()
smn.requires_grad = True
%time backbone = fastbuild.MakeBackbone.apply(smn.clone())
s = backbone.sum()
%time s.backward()

In [ ]:
diff = (goodgrad-smn.grad).detach().numpy()

In [ ]:
err = (diff*diff).sum(axis=(1,2))

In [ ]:
plt.plot(err)
err.max()

In [ ]:
smn = ncac.clone().detach().double()
smn.requires_grad = True
%time backbone = fastbuild.make_backbone_(smn.clone())
s = backbone.sum()
%time s.backward()
goodgrad = smn.grad

In [ ]:
smn = ncac.clone().detach().double()
smn.requires_grad = True
%time backbone = fastbuild.MakeBackbone.apply(smn.clone())
s = backbone.sum()
%time s.backward()

In [ ]:
diff = (goodgrad-smn.grad).detach().numpy()
err = (diff*diff).sum(axis=(1,2))
plt.plot(err)
err.max()